In [1]:
import matplotlib.pyplot as plt

print("Begin init")
%run -i notebook/init_notebook.ipynb


import pandas as pd
import numpy as np

from structlog import get_logger
from tqdm import tqdm


from src.model.StockModel import StockModel
from src.repository.FileOhlcRepository import FileOhlcRepository
from src.repository.SymbolRepository import SymbolRepository
from src.scrapper.YahooScrapper import YahooScrapper
from datetime import datetime, timedelta
print("Loading modules complete", datetime.now())

Begin init


Loading modules complete 2021-05-06 11:13:37.347550


In [2]:
def _get_1min_data(symbol):
    from structlog import get_logger
    logger = get_logger()
    stop_date = datetime.utcnow()
    start_date = stop_date - timedelta(days=365)
    ohlc_repository = FileOhlcRepository("./_data/", logger)
    df_original = ohlc_repository.read_data("yahoo", symbol, start_date, stop_date)

    df_original = df_original[df_original['volume'] > 0]
    return df_original


stock = StockModel.parse("NASDAQ:MSFT")
min1_df = _get_1min_data(stock)
min1_df.describe()

,low,close,open,high,volume
count,75406.000000,75406.000000,75406.000000,75406.000000,7.540600e+04
mean,222.795352,222.898161,222.898428,222.999701,3.104025e+09
std,15.830188,15.823068,15.822772,15.815738,8.562049e+10
min,196.380000,196.470000,196.455000,196.670000,1.420000e+02
25%,210.921250,211.045250,211.050000,211.150000,3.107000e+04
50%,216.920000,217.029500,217.030000,217.125000,4.727600e+04
75%,233.731500,233.850000,233.860000,233.960000,7.585350e+04
max,262.950000,263.180000,263.190000,263.190000,5.425764e+12


In [3]:
from src.calulation.BollingerBand import BollingerBand
from src.calulation.PpoBuilder import PpoBuilder
import src.calulation.utils as ta_utils

from src.calulation.indicator_builder import build_indicatrors

rocs = [2, 5, 10] # 2, 5, 10
property = 'close'
indicators = [
    # ta.VortexBuilder(14, rocs=rocs),
    # ta.VortexBuilder(24, rocs=rocs),
    # ta.VortexBuilder(32, rocs=rocs),
    #
    # ta.RsiBuilder(14, 8,  rocs=rocs),
    # ta.RsiBuilder(24, 8,  rocs=rocs),
    # ta.RsiBuilder(32, 8,  rocs=rocs),`


    # BollingerBand(property=property, n=14, rocs=rocs),
    # BollingerBand(property=property, n=24, rocs=rocs),
    # BollingerBand(property=property, n=32, rocs=rocs),

    #PpoBuilder(slow=30, fast=10, smooth=5, rocs=rocs)
]

df = min1_df.copy()
""
basic_price = df.iloc[0].low
basic_volume = df.volume.min()
ta_utils.normilize_data_(df, basic_price, basic_volume)


min1_df_with_indicators = build_indicatrors(df, indicators)
min1_df_with_indicators = min1_df_with_indicators.iloc[2000:-1] # drop warmup

# print(min1_df_with_indicators.iloc[-1].to_dict())
raw_columns = {'date', 'low', 'close', 'high', 'open', 'volume'}
for col in min1_df_with_indicators.columns:
    if col in raw_columns:
        continue

    print(f"{col}")
    plt.title(col)
    min1_df_with_indicators[col].hist(bins=100)
    plt.show()
    print(f"Min:{min1_df_with_indicators[col].min()}")
    print(f"Max:{min1_df_with_indicators[col].max()}")
    abs = np.abs(min1_df_with_indicators[col])
    print()
    print("75%", np.percentile(abs, 75))
    print("80%", np.percentile(abs, 80))
    print("90%", np.percentile(abs, 99))
    print("99%", np.percentile(abs, 99))
    print("*"*200)


